In [0]:
%tensorflow_version 2.x
# !pip install --upgrade tensorflow
# !pip install --upgrade tensorflow-hub
import tensorflow as tf
import tensorflow_hub as hub
tf.__version__,hub.__version__

TensorFlow 2.x selected.


('2.0.0', '0.7.0')

In [20]:
#Let us get embeddings of few terms and get their Cosine Similiarity from precomputed Module.
import tensorflow_hub as hub
import pandas as pd
import numpy as np
hub_url="https://tfhub.dev/google/nnlm-en-dim50/2"
embed = hub.KerasLayer(hub_url,trainable=True)
test=['hadoop','hive','.net']
embeddings = embed(test)
print(embeddings.shape, embeddings.dtype,type(embeddings))
embeddings_numpy=embeddings.numpy()
doc_embed = pd.DataFrame(data=embeddings_numpy)
doc_embed.index=test

def get_similiarity(target_text_embed,text_to_compare_embed):
    from sklearn.metrics.pairwise import cosine_similarity
    similiarity=cosine_similarity(target_text_embed,text_to_compare_embed)
    similiarity=pd.DataFrame(similiarity)
    return similiarity
sim=get_similiarity(doc_embed,doc_embed)
np.fill_diagonal(sim.values, 0)
sim.index=test
sim.columns=test
y=sim.idxmax(axis=1).values.tolist()
x=test
z=sim.max(axis=1).values.tolist()
pd.set_option('display.max_colwidth', -1)
data_global=pd.DataFrame([x,y,z]).T
data_global

(3, 50) <dtype: 'float32'> <class 'tensorflow.python.framework.ops.EagerTensor'>


,0,1,2
0,hadoop,hive,0.0503667
1,hive,.net,0.137715
2,.net,hive,0.137715


In [0]:
#Let us fine tune using a simple objective function of classification, to bring two words together.

#Caution: Please note the other weights also change during fine tuning loosing generalization. Play with Caution!


import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras


module_obj=hub.load("https://tfhub.dev/google/nnlm-en-dim50/2")
hub_layer=hub.KerasLayer(module_obj,trainable=True,input_shape=[], dtype=tf.string)

model = keras.Sequential()
model.add(hub_layer)
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy')

text_values_list=['hive','hadoop','.net']
labels=[0,0,1]
model.fit(x=text_values_list,y=labels,epochs=50)
model.summary()

import os
os.makedirs("finetuned_module_export", exist_ok=True)
export_module_dir = os.path.join(os.getcwd(), "finetuned_module_export")
tf.saved_model.save(module_obj, export_module_dir)

print (export_module_dir)

Train on 3 samples
Epoch 1/50
3/3 [==============================] - 1s 236ms/sample - loss: 0.7089
Epoch 2/50
3/3 [==============================] - 0s 93ms/sample - loss: 0.6858
Epoch 3/50
3/3 [==============================] - 0s 93ms/sample - loss: 0.6704
Epoch 4/50
3/3 [==============================] - 0s 94ms/sample - loss: 0.6578
Epoch 5/50
3/3 [==============================] - 0s 94ms/sample - loss: 0.6471
Epoch 6/50
3/3 [==============================] - 0s 95ms/sample - loss: 0.6362
Epoch 7/50
3/3 [==============================] - 0s 92ms/sample - loss: 0.6261
Epoch 8/50
3/3 [==============================] - 0s 93ms/sample - loss: 0.6167
Epoch 9/50
3/3 [==============================] - 0s 93ms/sample - loss: 0.6076
Epoch 10/50
3/3 [==============================] - 0s 93ms/sample - loss: 0.5968
Epoch 11/50
3/3 [==============================] - 0s 92ms/sample - loss: 0.5866
Epoch 12/50
3/3 [==============================] - 0s 92ms/sample - loss: 0.5776
Epoch 13/50
3/3 [

INFO:tensorflow:Assets written to: /content/finetuned_module_export/assets


/content/finetuned_module_export


In [0]:
import tensorflow_hub as hub
import pandas as pd
import numpy as np
print ("export_module_dir",export_module_dir)
hub_url=export_module_dir
embed = hub.KerasLayer(hub_url)
test=['hadoop','hive','.net']
embeddings = embed(test)
print(embeddings.shape, embeddings.dtype,type(embeddings))
embeddings_numpy=embeddings.numpy()
doc_embed = pd.DataFrame(data=embeddings_numpy)
doc_embed.index=test

def get_similiarity(target_text_embed,text_to_compare_embed):
    from sklearn.metrics.pairwise import cosine_similarity
    similiarity=cosine_similarity(target_text_embed,text_to_compare_embed)
    similiarity=pd.DataFrame(similiarity)
    return similiarity
sim=get_similiarity(doc_embed,doc_embed)
np.fill_diagonal(sim.values, 0)
# idx=sim.idxmax(axis=1).values[0]
# sim[idx]

sim.index=test
sim.columns=test
y=sim.idxmax(axis=1).values.tolist()
x=test
z=sim.max(axis=1).values.tolist()
pd.set_option('display.max_colwidth', -1)
data_finetuned=pd.DataFrame([x,y,z]).T
# data.to_csv('u3_u4.csv')
data_finetuned

export_module_dir /content/finetuned_module_export
(3, 50) <dtype: 'float32'> <class 'tensorflow.python.framework.ops.EagerTensor'>


,0,1,2
0,hadoop,hive,0.13457
1,hive,hadoop,0.13457
2,.net,hive,0.00266996


In [21]:
data_finetuned.columns=['word','Fine tuned most similiar','Finetuned_similiarity']
data_global.columns=['word','Global most similiar','Global similiarity']
data_global.merge(data_finetuned,how='left',on='word')

,word,Global most similiar,Global similiarity,Fine tuned most similiar,Finetuned_similiarity
0,hadoop,hive,0.0503667,hive,0.13457
1,hive,.net,0.137715,hadoop,0.13457
2,.net,hive,0.137715,hive,0.00266996
